In [207]:
import pandas as pd
import numpy as np
import json
import math

ANGLE_W = 1
SPEED_W = 15

with open("./maps/path/to/level.dat") as json_data:
    data = json.load(json_data)
notes = pd.DataFrame(data['_notes'])

In [208]:
# Calculate note centers
notes['_yCenter'] = notes['_lineLayer'].apply(lambda x: 1 + x * 0.55)
notes['_xCenter'] = notes['_lineIndex'].apply(lambda x: -0.9 + x * 0.6)

# Split to left and right handed notes
left = notes[notes['_type'] == 0].copy()
right = notes[notes['_type'] == 1].copy()

In [209]:
# Calculate position and angle changes between notes
# Each row contains the change between the previous note and itself
for df in [left, right]:
    df['_xMovement'] = df['_xCenter'].diff()
    df['_yMovement'] = df['_yCenter'].diff()
    df['_totMovement'] = np.hypot(df['_xMovement'], df['_yMovement'])
    df['_angleChange'] = np.arctan(df['_yMovement'] / df['_xMovement'])
    df['_timeChange'] = df['_time'].diff()
    df[:1] = df[:1].fillna(0) # Set movements and changes to 0 in first row

In [210]:
average_angle = (left['_angleChange'].abs().mean() + right['_angleChange'].abs().mean()) / 2

sum_of_swing_times = left[left['_timeChange'] < 5]['_timeChange'].sum() + right[right['_timeChange'] < 5]['_timeChange'].sum() # In beats
total_distance = left[left['_totMovement'] > 0.54]['_totMovement'].sum() + right[right['_totMovement'] > 0.54]['_totMovement'].sum()
average_speed = total_distance / sum_of_swing_times

complexity = ANGLE_W * average_angle + SPEED_W * average_speed

print(complexity)

15.851200674253983
